In [1]:
import pandas as pd
import os
import glob
try:
    code_dir
except NameError:
    code_dir = os.getcwd()
    base_dir = code_dir.replace("/codes_local", "")
    deseq2_dir = base_dir + "/3_DEseq2"
    deseq2_compile_dir = base_dir + "/3_DEseq2_compiled"
base_dir

'/media/pipkin/Yolanda/Exp337CD25KONascent_new'

# TPM

In [2]:
ref_gff = base_dir + '/codes_hpc/Mus_musculus.GRCm38.102.mRNA.simp.rmdup.srt.gff3'
ref_gff_df = pd.read_csv(ref_gff, sep="\t", header=None)
ref_gff_df['gene_name'] = [x.replace("ID=", "") for x in ref_gff_df[8]]
ref_gff_df['len'] = ref_gff_df[4] - ref_gff_df[3]
ref_gff_df['len_kb'] = ref_gff_df['len'] / 1000
ref_gff_df = ref_gff_df[['gene_name', 'len_kb']]
ref_gff_df = ref_gff_df.set_index('gene_name')

In [29]:
###----- Convert raw count to tpm
raw_count_file = base_dir + '/2_featureCounts_merged/Exp337_count.csv'
raw_count_df = pd.read_csv(raw_count_file, index_col=0)
ref_gff_df_matched = ref_gff_df.loc[raw_count_df.index.tolist()] # Match gene name order
#--- Calculate RPK (read counts divide by length of gene in kilobases)
rpk_df = raw_count_df.transpose() / ref_gff_df_matched['len_kb'].tolist()
rpk_df = rpk_df.transpose()
#--- Calculate scaling factor (sum up RPK values and divide by 1,000,000)
scaling_factor = list(rpk_df.sum() / 1000000)
#--- Adjust rpk by scalling factor
tpm_df = rpk_df / scaling_factor

# Ordered conditions
conds = []
for i in [0,6,24,48]:
    conds += ["WT_%sh_rep1"%i, "WT_%sh_rep2"%i, "WT_%sh_rep3"%i]
    conds += ["KO_%sh_rep1"%i, "KO_%sh_rep2"%i, "KO_%sh_rep3"%i]
conds = [x for x in conds if x in tpm_df.columns.tolist()]

tpm_file = deseq2_compile_dir + "/tpm_bysample.csv"
tpm_df = tpm_df[conds]
tpm_df.round(3).to_csv(tpm_file)

In [30]:
###----- Averge tpm per condition
# Ordered conditions
conds = []
for i in [0,6,24,48]:
    conds.append("WT_%sh"%i)
    conds.append("KO_%sh"%i)

tpm_df_t = tpm_df.transpose()
tpm_df_t['cond'] = [x.split("_rep")[0] for x in tpm_df_t.index]
avg_tpm_df = tpm_df_t.groupby('cond').mean().transpose()
avg_tpm_df = avg_tpm_df[conds]

tpm_file = deseq2_compile_dir + "/tpm.csv"
avg_tpm_df.round(3).to_csv(tpm_file)

# Normalized count

In [20]:
###----- Calculate averge normalized count
norm_count_file = deseq2_compile_dir + "/DESeq2_normalized_counts_bysample.csv"
avg_norm_count_file = deseq2_compile_dir + "/DESeq2_normalized_counts.csv"

norm_count_df = pd.read_csv(norm_count_file, index_col=0)
all_genes = norm_count_df.index.tolist()

# Create empty df for average normalized count
avg_norm_df = pd.DataFrame({"gene_name": all_genes}).set_index("gene_name")
conds = []
for i in [0,6,24,48]:
    conds.append("WT_%sh"%i)
    conds.append("KO_%sh"%i)
# Calculate averge for each condition    
for cond in conds:
    cond_reps = [x for x in norm_count_df.columns if cond+"_rep" in x]
    cond_df = norm_count_df[cond_reps]
    avg_norm_df[cond] = cond_df.mean(axis=1).tolist()
avg_norm_df.round(3).to_csv(avg_norm_count_file)

# DESEQ outputs

In [45]:
###----- Compile DEseq output
deseq2_files = glob.glob("%s/*.csv"%deseq2_dir)

all_genes = pd.read_csv(deseq2_files[0])['gene_name']
all_genes = [x for x in all_genes if str(x) != 'nan']
all_genes = list(set(all_genes))

for i in ['log2FoldChange','stat', 'pvalue', 'padj']:
    i_df = pd.DataFrame({"gene_name": all_genes})
    for deseq2_file in deseq2_files:
        deseq2_file_name_simp = deseq2_file.split("/")[-1].replace(".csv","")
        deseq2_df = pd.read_csv(deseq2_file)
        deseq2_df.columns = ["gene_name"] + deseq2_df.columns.tolist()[1:]
        deseq2_i_df = deseq2_df[['gene_name', i]]
        deseq2_i_df.columns = ['gene_name', deseq2_file_name_simp]
        i_df = i_df.merge(deseq2_i_df, how='left')
    i_df_name = deseq2_compile_dir + "/DESeq2_%s.csv"%i
    i_df.to_csv(i_df_name, index=False)